In [1]:
import numpy as np

import torch
import torch.nn as nn
import torchvision

from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
])

dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/multiclass-weather-dataset/Multi-class Weather Dataset', transform=transform)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)


In [2]:
class ModelMaker():
    def __init__(self):
        pass
        
    def model_extractor(self):
        resnet = torchvision.models.resnet18()

        resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

        resnet.eval()

        train_features = []
        train_labels = []
        for images, labels in train_dataset:
            with torch.no_grad():
                features = resnet(images.unsqueeze(0))
            train_features.append(features.squeeze().numpy())
            train_labels.append(labels)

        test_features = []
        test_labels = []
        for images, labels in test_dataset:
            with torch.no_grad():
                features = resnet(images.unsqueeze(0))
            test_features.append(features.squeeze().numpy())
            test_labels.append(labels)
            
        train_features = np.array(train_features)
        train_labels = np.array(train_labels)
        test_features = np.array(test_features)
        test_labels = np.array(test_labels)
        
        return train_features, train_labels, test_features, test_labels
        

In [3]:
MM = ModelMaker()
train_features, train_labels, test_features, test_labels = MM.model_extractor()

In [4]:
def Accuracy_Calculator(classifier, name):
    classifier.fit(train_features, train_labels)

    predictions = classifier.predict(test_features)

    accuracy = accuracy_score(test_labels, predictions)
    
    print("###################################################################################################################")
    print(f"# Classification Performance for {name}: {accuracy}")
    print("###################################################################################################################")

In [5]:
Accuracy_Calculator(LinearSVC(dual=False), "Support Vector Machine (SVM)")

###################################################################################################################
# Classification Performance for Support Vector Machine (SVM): 0.9244444444444444
###################################################################################################################


In [6]:
Accuracy_Calculator(LogisticRegression(max_iter=3000), "Logistic Regression (LR)")

###################################################################################################################
# Classification Performance for Logistic Regression (LR): 0.8844444444444445
###################################################################################################################


In [7]:
Accuracy_Calculator(KNeighborsClassifier(n_neighbors=5), "K Nearest Neighbour (KNN)")

###################################################################################################################
# Classification Performance for K Nearest Neighbour (KNN): 0.7688888888888888
###################################################################################################################
